In [0]:
import pandas as pd

start_from_pickle = True

In [0]:
import pickle

if not start_from_pickle:
    with open(
        "/Workspace/Users/wilson.kan@neofinancial.com/ApplicationV2/pkls/appl_tim_v2_modeling_ready.pkl",
        "wb",
    ) as f:  # open a text file
        pickle.dump(modeling_dummy_df, f)
else:
    with open(
        "/Workspace/Users/wilson.kan@neofinancial.com/ApplicationV2/pkls/appl_tim_v2_modeling_ready.pkl",
        "rb",
    ) as f:  # Correctly opening the file in binary read mode
        modeling_dummy_df = pickle.load(f)

In [0]:
import numpy as np

# Due to data availability 5 fold validation is used over out of time testing

modeling_intime = modeling_dummy_df[
    modeling_dummy_df["month_end"] <= np.datetime64("2024-03-31")
]

In [0]:
# Modeling only subprime

modeling_intime = modeling_intime[modeling_intime["subprime"] == 1]

In [0]:
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split

# dropping duplicated features
X_train, X_test, y_train, y_test = train_test_split(
    modeling_intime.drop(
        ["month_end", "isdefault_1y", "originalCreditScore", "GO17"], axis=1
    ),
    modeling_intime["isdefault_1y"],
    test_size=0.2,
    random_state=825147,
)
# create model instance
bst = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    colsample_bytree=0.75,
    subsample=0.5,
    gamma=1,
    eta=0.1,
    min_child_weight=2,
    objective="binary:logistic",
    random_state=250693,
)
# fit model
bst.fit(
    X_train,
    y_train,
    verbose=True,
    early_stopping_rounds=10,
    eval_metric="auc",
    eval_set=[(X_test, y_test)],
)
xgb.plot_importance(bst)

In [0]:
feature_imp = []

for i in range(len(bst.feature_importances_)):
    feature_imp.append((bst.feature_names_in_[i], bst.feature_importances_[i]))

feature_imp.sort(key=lambda tup: tup[1], reverse=True)
feature_imp

In [0]:
# # Only need to run once
# # Optimizing the number of features to use - 16 was selected

# from sklearn.metrics import roc_auc_score
# import warnings

# warnings.filterwarnings("ignore")

# def range_test(num_col):
#   top_set = []
#   for i in range(num_col):
#       top_set.append(feature_imp[i][0])

#   # create model instance
#   bst = XGBClassifier(
#       n_estimators=100,
#       max_depth=6,
#       colsample_bytree=0.75,
#       subsample=0.5,
#       gamma=1,
#       eta=0.1,
#       min_child_weight=3,
#       random_state=445287,
#   )
#   # fit model
#   bst.fit(
#       X_train[top_set],
#       y_train,
#       early_stopping_rounds=10,
#       eval_metric="auc",
#       eval_set=[(X_test[top_set], y_test)],
#       verbose=False
#   )

#   x_train_scr = bst.predict_proba(X_train[top_set])
#   x_test_scr = bst.predict_proba(X_test[top_set])
#   return (2 * roc_auc_score(y_train, x_train_scr[:, 1]) - 1, 2 * roc_auc_score(y_test, x_test_scr[:, 1]) - 1)

# res = {"n": [], "train_gini": [], "test_gini": []}
# for i in range(10, len(feature_imp)):
#   rt = range_test(i)
#   res["n"].append(i)
#   res["train_gini"].append(rt[0])
#   res["test_gini"].append(rt[1])

# res = pd.DataFrame(res)
# res.plot(x="n", y=["train_gini", "test_gini"])
# res.to_csv("/Workspace/Users/wilson.kan@neofinancial.com/ApplicationV2/Misc/tims_subprime_feature_cnt.csv")

In [0]:
top_set = []
for i in range(16):
    top_set.append(feature_imp[i][0])
print(top_set)
# create model instance
bst = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    colsample_bytree=0.75,
    subsample=0.5,
    gamma=1,
    eta=0.1,
    min_child_weight=3,
    random_state=445287,
)
# fit model
bst.fit(
    X_train[top_set],
    y_train,
    early_stopping_rounds=10,
    eval_metric="auc",
    eval_set=[(X_test[top_set], y_test)],
)
xgb.plot_importance(bst)

In [0]:
from sklearn.metrics import roc_auc_score

x_train_scr = bst.predict_proba(X_train[top_set])
x_test_scr = bst.predict_proba(X_test[top_set])
print("train", 2 * roc_auc_score(y_train, x_train_scr[:, 1]) - 1)
print("test", 2 * roc_auc_score(y_test, x_test_scr[:, 1]) - 1)

In [0]:
sp = modeling_dummy_df[modeling_dummy_df["subprime"] == 1]
x_subprime_scr = bst.predict_proba(sp[top_set])
print("subprime", 2 * roc_auc_score(sp["isdefault_1y"], x_subprime_scr[:, 1]) - 1)

In [0]:
feature_imp = []

for i in range(len(bst.feature_importances_)):
    feature_imp.append((bst.feature_names_in_[i], bst.feature_importances_[i]))

feature_imp.sort(key=lambda tup: tup[1], reverse=True)
for (k, v) in feature_imp:
    print(f"{k}\t{v}")

## Sensitivity Testing to check for direction and impact
Sentivity done with intime data instead of oot data due to data limitation

In [0]:
def sensitivity(model, dat, var_set, var_type, effect):
    res = {}
    for v in var_set:
        if var_type == "int":
            dat.loc[:, v] = dat[v] + effect
        elif var_type == "cont":
            dat.loc[:, v] = dat[v] * effect
        elif var_type == "bin":
            dat.loc[:, v] = effect
        elif var_type == "orig":
            dat.loc[:, v] = dat[v]
        else:
            raise Exception("Invalid variable type")
        res[v] = model.predict_proba(dat)[:, 1].mean()
    return res

In [0]:
int_var = ['RE03',
'GO151',
'GO148',
'GO15',
'RE37']
cont_var = ['AT60',
'BC60',
'BC34',
'AT34',
'AT21',
'BC21',
'GO11',
'GO07',
'RE62',
'BR62',
'AM04']
bin_var = []

In [0]:
vani = sensitivity(bst, modeling_intime[top_set], [int_var[0]], "orig", 1)[int_var[0]]
int_inc = sensitivity(bst, modeling_intime[top_set], int_var, "int", 3)
int_des = sensitivity(bst, modeling_intime[top_set], int_var, "int", -3)
cont_inc = sensitivity(bst, modeling_intime[top_set], cont_var, "cont", 1.3)
cont_des = sensitivity(bst, modeling_intime[top_set], cont_var, "cont", 0.7)
inc = int_inc | cont_inc
des = int_des | cont_des

In [0]:
final_top_set = [i[0] for i in feature_imp]
sens_table = {"vanilla": [], "feat_inc": [], "feat_des": []}
for v in final_top_set:
    sens_table["vanilla"].append(vani)
    sens_table["feat_inc"].append(inc[v])
    sens_table["feat_des"].append(des[v])
sens_out = pd.DataFrame(sens_table, index=final_top_set)
sens_out

##Test new model with strange direction removed (iterative process)

In [0]:
top_set_refined = [
    v for v in top_set if v not in [
"BC60",
"BC34",
'GO11',
"AT34",
'GO15',
'GO151',
"GO148"]
]

# top_set_refined.extend(["AT84"])

In [0]:
# create model instance
bst = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    colsample_bytree=0.75,
    subsample=0.5,
    gamma=1,
    eta=0.1,
    min_child_weight=2,
    random_state=150691,
)
# fit model
bst.fit(
    X_train[top_set_refined],
    y_train,
    early_stopping_rounds=10,
    eval_metric="auc",
    eval_set=[(X_test[top_set_refined], y_test)],
)
xgb.plot_importance(bst)

In [0]:
x_train_scr = bst.predict_proba(X_train[top_set_refined])
x_test_scr = bst.predict_proba(X_test[top_set_refined])
print("train", 2 * roc_auc_score(y_train, x_train_scr[:, 1]) - 1)
print("test", 2 * roc_auc_score(y_test, x_test_scr[:, 1]) - 1)

x_subprime_scr = bst.predict_proba(sp[top_set_refined])
print("subprime", 2 * roc_auc_score(sp["isdefault_1y"], x_subprime_scr[:, 1]) - 1)

feature_imp_refined = []

for i in range(len(bst.feature_importances_)):
    feature_imp_refined.append((bst.feature_names_in_[i], bst.feature_importances_[i]))

feature_imp_refined.sort(key=lambda tup: tup[1], reverse=True)

In [0]:
int_var = ['RE03',
'AM04',
'RE37']
cont_var = [
'BC21',"AT60",
'RE62',
'GO07',
'BR62',"AT21"]
bin_var = []

vani = sensitivity(bst, modeling_intime[top_set_refined], [int_var[0]], "orig", 1)[
    int_var[0]
]
int_inc = sensitivity(bst, modeling_intime[top_set_refined], int_var, "int", 1)
int_des = sensitivity(bst, modeling_intime[top_set_refined], int_var, "int", -1)
cont_inc = sensitivity(bst, modeling_intime[top_set_refined], cont_var, "cont", 1.3)
cont_des = sensitivity(bst, modeling_intime[top_set_refined], cont_var, "cont", 0.7)
inc = int_inc | cont_inc
des = int_des | cont_des

final_top_set = [i[0] for i in feature_imp_refined]
sens_table = {"vanilla": [], "feat_inc": [], "feat_des": []}
for v in final_top_set:
    sens_table["vanilla"].append(vani)
    sens_table["feat_inc"].append(inc[v])
    sens_table["feat_des"].append(des[v])
sens_out = pd.DataFrame(sens_table, index=final_top_set)
sens_out

#5 fold validation

In [0]:
import random

random.seed(46993)

# Label data
modeling_intime["fold"] = [random.randint(1, 5) for i in range(len(modeling_intime))]

for i in range(1, 6):
    model_cut = modeling_intime[modeling_intime["fold"] == i]
    X_cut = model_cut[top_set_refined]
    y_cut = model_cut["isdefault_1y"]
    x_cut_scr = bst.predict_proba(X_cut)
    print(f"gini cut {i}: {2 * roc_auc_score(y_cut, x_cut_scr[:, 1]) - 1}")

## Save model that can be loaded later with load_model

In [0]:
# bst.save_model(
#     "/Workspace/Users/wilson.kan@neofinancial.com/ApplicationV2/SaveModels/tims_subprime.json"
# )